## Modified with test-wise questions and marking scheme data

Steps:
- Select a student with specific ability (Concept-wise) and cohort of the student.
- Get test with some questions and marking system.
- Get the data of questions parameters.
- Based on concept-wise ability and questions parameters find P(theta) for the questions from the test.
- Using the threshold for P(theta) for student's cohort find the correctly solved questions (prediction).
- Using the marking system find the score of selected student in given test.

Data files used:
- Ability Data: Concept wise ability_(test more than 4).csv
- 3PL parameters: Paramaeters_3pl_ba(c in 0-1)_all.csv
- Unique test ids files (Serious attempts): Unique Test Ids.xslx
- Students raw data
- Optimal Ptheat (cohort wise): Cohortwise Optimal Ptheta.xlsx
- Test-wise marking scheme data: Test wise marking scheme.xlsx

In [1]:
# Loading Packages
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load 3PL model parameter's data
p1 = 'D:/iAnalyst/MyPat task/shared/'
bac_param = pd.read_csv(p1 + 'Paramaeters_3pl_ba(c in 0-1)_all.csv')
bac_param.columns = ['Question ID','b','a','c']
bac_param.head()

,Question ID,b,a,c
0,5c0e37947af61505b3cbc9fc,0.473189,1.044653,0.249297
1,5c0e397f7af61505b3cbca36,0.881615,1.505793,0.204460
2,5c0e3b197af61505b3cbcae7,-0.041069,0.959467,0.224513
3,5c0e3cfc7af61505b3cbcc25,-0.071553,0.950171,0.223157
4,5c0e3e887af61505b3cbcc76,0.490655,1.053462,0.250685


In [3]:
test_marks_df = pd.read_excel(p1+'Test wise marking scheme.xlsx')
test_marks_df.columns = ['Test ID', 'Question ID','PositiveMarks',
                        'NegativeMarks','PartialMarks']
test_marks_df.head()

,Test ID,Question ID,PositiveMarks,NegativeMarks,PartialMarks
0,5ace202b9fe7b01d5806ba0d,5a8ad2a8738197281fbac941,3.0,1.0,0.0
1,5ace202b9fe7b01d5806ba0d,5a8ad2a8738197281fbac943,3.0,1.0,0.0
2,5ace202b9fe7b01d5806ba0d,5a8ad2a8738197281fbac945,3.0,1.0,0.0
3,5ace202b9fe7b01d5806ba0d,5a8ad2a8738197281fbac947,3.0,1.0,0.0
4,5ace202b9fe7b01d5806ba0d,5a8ad2a8738197281fbac949,3.0,1.0,0.0


In [4]:
# Load students data with concept wise ability
ability = pd.read_csv(p1 + 'Concept wise ability_(test more than 4).csv')
ability.head()

,Mapped Master Concept,1011101700001_Top1000_Theta,1011101700004_Top500_Theta,1011211700001_Top2000_Theta,1011211700003_Top25000_Theta,1011211700004_Top15000_Theta,1011221700003_Top15000_Theta,1011221700006_Top25000_Theta,1011291700002_Top25000_Theta,1011331700001_Top10000_Theta,...,9065731600021_Top15000_Theta,9065731600035_Top50000_Theta,9065731600077_Top5000_Theta,9065731600138_Top20000_Theta,9065741590008_Top25000_Theta,9065831590001_Top20000_Theta,9065831590002_Top10000_Theta,9065831590003_Top10000_Theta,9071101590001_Top5000_Theta,9072291590001_Top20000_Theta
0,5ace203e9fe7b01d5806c0de,1.466337,1.945910,0.510826,0.000000,-4.000000,0.510826,0.000000,NaN,0.000000,...,0.000000,-4.0,1.466337,-4.0,0.0,4.000000,0.000000,4.000000,0.000000,-0.693147
1,5ace20349fe7b01d5806bdba,0.000000,1.098612,-0.693147,NaN,-0.559616,0.000000,0.000000,-4.0,-0.405465,...,NaN,NaN,0.405465,-4.0,-4.0,4.000000,NaN,0.693147,NaN,0.000000
2,5ace20399fe7b01d5806bf50,4.000000,1.386294,1.098612,0.000000,4.000000,4.000000,1.609438,NaN,-0.405465,...,-0.693147,4.0,0.000000,NaN,-4.0,-4.000000,NaN,-4.000000,-4.000000,0.832909
3,5ace203f9fe7b01d5806c145,0.693147,0.000000,2.079442,NaN,1.609438,0.000000,0.405465,4.0,0.287682,...,0.000000,-4.0,-0.405465,NaN,0.0,-4.000000,NaN,-4.000000,4.000000,0.693147
4,5ace203f9fe7b01d5806c150,-0.788457,1.152680,0.693147,1.098612,0.087011,0.510826,-0.459532,-4.0,0.693147,...,0.693147,-4.0,0.000000,-4.0,-4.0,0.693147,0.980829,4.000000,-1.098612,0.693147


In [5]:
# Unique test ids files
test_ids = pd.read_excel(p1 + 'Unique Test Ids.xlsx')
test_ids.head()

,Test ID
0,5ace20a99fe7b01d58073902
1,5bbde060deb0c602d8fafee8
2,5c35c849f1f5e055cc0f3af3
3,5ace202c9fe7b01d5806ba9f
4,5ace20a39fe7b01d58073080


In [6]:
test_marks_df[test_marks_df['Test ID']=='5ace202c9fe7b01d5806ba9f'].shape

(54, 5)

In [63]:
# Unique test ids files
opt_ptheta = pd.read_excel(p1 + 'Cohortwise Optimal Ptheta.xlsx')
opt_ptheta.head()

,Cohort,Ptheta
0,Top10,0.65
1,Top20,0.60
2,Top50,0.57
3,Top100,0.58
4,Top200,0.59


In [19]:
def Test_score_predictor(df_param,df_test,df_ptheta,df_ability):
    c = ['Top10', 'Top20', 'Top50' ,'Top100', 'Top200', 'Top500', 'Top1000', 'Top1500', 'Top2000','Top5000' ,'Top10000', 'Top15000','Top20000','Top25000','Top50000']
    print(c,'\n')
    print('Select Cohort File From Above List...','\n')
    f = input('Cohort:')
    print('Loading Data For Selected Cohort...','\n')
    df_cohort = pd.read_csv(p1 + f + '.csv')
    df_cohort = df_cohort[df_cohort['Test ID'].isin(df_test['Test ID'])]
    print('Loading Completed...','\n')
    print(df_cohort.head())
    
    out = pd.DataFrame(columns=['Student ID','Test ID', 'Max Marks',
                               'My Marks (Actual)','My Marks (Predicted)',
                               'Deviation','% Deviation'])
    # Lets work on each enrolment numbers in selected cohort
    for en in df_cohort['Enrolment No'].unique():
        df_en = df_cohort[df_cohort['Enrolment No']==en]
        # Proceed if total tests attempted are more than 4
        if df_en['Test ID'].nunique() > 4:
            print(df_en.shape)
            ability_en = df_ability[['Mapped Master Concept',
                                     str(en)+'_'+f+'_'+'Theta']]
            #print(ability_en.head())
            df_merge1 = pd.merge(df_en,df_param,on = 'Question ID', how = 'inner')
            df_merge2 = pd.merge(df_merge1,df_test,on = ['Test ID','Question ID'], how = 'inner')
            df_merge3 = pd.merge(df_merge2,ability_en, on = 'Mapped Master Concept',
                                how = 'inner')
            print(df_merge3.head())
    return df_merge3

In [22]:
df= Test_score_predictor(bac_param,test_marks_df,opt_ptheta,ability)

['Top10', 'Top20', 'Top50', 'Top100', 'Top200', 'Top500', 'Top1000', 'Top1500', 'Top2000', 'Top5000', 'Top10000', 'Top15000', 'Top20000', 'Top25000', 'Top50000'] 

Select Cohort File From Above List... 

Cohort:Top10
Loading Data For Selected Cohort... 

Loading Completed... 

    Enrolment No  AIR               Question ID     Mapped Master Concept  \
0  9062181600080    5  5bb59ce843750164ad7c9dcf  5ace20399fe7b01d5806bf5a   
1  9062181600080    5  5bb59ddf43750164ad7c9de6  5ace20399fe7b01d5806bf5a   
2  9062181600080    5  5bb59e3443750164ad7c9df0  5ace20399fe7b01d5806bf5a   
3  9062181600080    5  5bb59f2743750164ad7c9e1e  5ace20399fe7b01d5806bf5a   
4  9062181600080    5  5bb59f7843750164ad7c9e33  5ace20399fe7b01d5806bf5a   

                    Test ID  Response     Time cohort  
0  5bb5ad1043750164ad7ca12c         0  297.053  Top10  
1  5bb5ad1043750164ad7ca12c         1   39.152  Top10  
2  5bb5ad1043750164ad7ca12c         1   48.641  Top10  
3  5bb5ad1043750164ad7ca12c        

In [13]:
df_10 = pd.read_csv(p1 + 'Top10' + '.csv')
df_10.head()

,Enrolment No,AIR,Question ID,Mapped Master Concept,Test ID,Response,Time,cohort
0,9062181600080,5,5bb59ce843750164ad7c9dcf,5ace20399fe7b01d5806bf5a,5bb5ad1043750164ad7ca12c,0,297.053,Top10
1,9062181600080,5,5bb59ddf43750164ad7c9de6,5ace20399fe7b01d5806bf5a,5bb5ad1043750164ad7ca12c,1,39.152,Top10
2,9062181600080,5,5bb59e3443750164ad7c9df0,5ace20399fe7b01d5806bf5a,5bb5ad1043750164ad7ca12c,1,48.641,Top10
3,9062181600080,5,5bb59f2743750164ad7c9e1e,5ace20399fe7b01d5806bf5a,5bb5ad1043750164ad7ca12c,1,160.302,Top10
4,9062181600080,5,5bb59f7843750164ad7c9e33,5ace20399fe7b01d5806bf5a,5bb5ad1043750164ad7ca12c,0,226.225,Top10


In [18]:
df_10['Enrolment No'].nunique()

5

In [16]:
[i for i in range(10)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [33]:
#Formula of IRT #3PL : https://en.wikipedia.org/wiki/Item_response_theory#:~:text=(The%20expression%20%E2%80%9Ca%20mathematical%20function,the%20person%20in%20their%20environment.)&text=In%20the%20same%20manner%2C%20IRT,behaviour%20in%20online%20social%20networks.

df['P_theta'] = df['c']+ (1-df['c'])/(1+(np.e)**(-df['a']* (df['1061921690009_Top10_Theta'] - df['b'])))
#df

,Enrolment No,AIR,Question ID,Mapped Master Concept,Test ID,Response,Time,cohort,b,a,c,PositiveMarks,NegativeMarks,PartialMarks,1061921690009_Top10_Theta,P_theta
0,1061921690009,2,5cde3610d8f0346457c618f9,5ace20419fe7b01d5806c1dc,5cde807ad8f0346457c62f9b,2,459.615,Top10,0.496548,1.055505,0.250689,3.0,1.0,0.0,1.098612,0.740537
1,1061921690009,2,5cde5138d8f0346457c61e1c,5ace20419fe7b01d5806c1dc,5cde807ad8f0346457c62f9b,1,127.960,Top10,0.479637,1.051228,0.250501,3.0,1.0,0.0,1.098612,0.743045
2,1061921690009,2,5c208e182d535234640dfd39,5ace20419fe7b01d5806c1dc,5c2083382d535234640dfa93,1,56.867,Top10,0.407088,1.024912,0.247393,4.0,1.0,0.0,1.098612,0.751734
3,1061921690009,2,5cde3610d8f0346457c618f9,5ace203f9fe7b01d5806c128,5cde807ad8f0346457c62f9b,2,459.615,Top10,0.496548,1.055505,0.250689,3.0,1.0,0.0,0.000000,0.529352
4,1061921690009,2,5c2451e02d535234640e3e66,5ace203f9fe7b01d5806c128,5c2472bb2d535234640e45ba,1,81.477,Top10,0.485072,1.048384,0.249594,4.0,1.0,0.0,0.000000,0.531398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,1061921690009,2,5c208aa62d535234640dfc2a,5ace20369fe7b01d5806be7a,5c2083382d535234640dfa93,1,12.116,Top10,0.388105,1.019618,0.246769,4.0,1.0,0.0,4.000000,0.981518
600,1061921690009,2,5c271b352d535234640e7dd6,5ace203d9fe7b01d5806c094,5c2995292d535234640e8681,1,24.993,Top10,0.513416,1.057459,0.250248,4.0,1.0,0.0,4.000000,0.981678
601,1061921690009,2,5c208ab72d535234640dfc42,5ace20349fe7b01d5806bdb4,5c2083382d535234640dfa93,1,47.595,Top10,0.505754,1.053767,0.250664,4.0,1.0,0.0,4.000000,0.981603
602,1061921690009,2,5c271cbd2d535234640e7e20,5ace203d9fe7b01d5806c08b,5c2995292d535234640e8681,1,75.350,Top10,0.496548,1.055505,0.250689,4.0,1.0,0.0,4.000000,0.981884


In [68]:
#opt_ptheta[opt_ptheta['Cohort']=='Top10'].Ptheta.values[0]

0.65

In [70]:
df["Predicted Responses"] = 0
for i in range(len(df['P_theta'])):
    if df['P_theta'][i] > opt_ptheta[opt_ptheta['Cohort']=='Top10'].Ptheta.values[0]:  #put 1 when 
        
        df['Predicted Responses'][i]= 1
        i=i+1
df

,Enrolment No,AIR,Question ID,Mapped Master Concept,Test ID,Response,Time,cohort,b,a,c,PositiveMarks,NegativeMarks,PartialMarks,1061921690009_Top10_Theta,P_theta,Predicted Responses
0,1061921690009,2,5cde3610d8f0346457c618f9,5ace20419fe7b01d5806c1dc,5cde807ad8f0346457c62f9b,2,459.615,Top10,0.496548,1.055505,0.250689,3.0,1.0,0.0,1.098612,0.740537,1
1,1061921690009,2,5cde5138d8f0346457c61e1c,5ace20419fe7b01d5806c1dc,5cde807ad8f0346457c62f9b,1,127.960,Top10,0.479637,1.051228,0.250501,3.0,1.0,0.0,1.098612,0.743045,1
2,1061921690009,2,5c208e182d535234640dfd39,5ace20419fe7b01d5806c1dc,5c2083382d535234640dfa93,1,56.867,Top10,0.407088,1.024912,0.247393,4.0,1.0,0.0,1.098612,0.751734,1
3,1061921690009,2,5cde3610d8f0346457c618f9,5ace203f9fe7b01d5806c128,5cde807ad8f0346457c62f9b,2,459.615,Top10,0.496548,1.055505,0.250689,3.0,1.0,0.0,0.000000,0.529352,0
4,1061921690009,2,5c2451e02d535234640e3e66,5ace203f9fe7b01d5806c128,5c2472bb2d535234640e45ba,1,81.477,Top10,0.485072,1.048384,0.249594,4.0,1.0,0.0,0.000000,0.531398,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,1061921690009,2,5c208aa62d535234640dfc2a,5ace20369fe7b01d5806be7a,5c2083382d535234640dfa93,1,12.116,Top10,0.388105,1.019618,0.246769,4.0,1.0,0.0,4.000000,0.981518,1
600,1061921690009,2,5c271b352d535234640e7dd6,5ace203d9fe7b01d5806c094,5c2995292d535234640e8681,1,24.993,Top10,0.513416,1.057459,0.250248,4.0,1.0,0.0,4.000000,0.981678,1
601,1061921690009,2,5c208ab72d535234640dfc42,5ace20349fe7b01d5806bdb4,5c2083382d535234640dfa93,1,47.595,Top10,0.505754,1.053767,0.250664,4.0,1.0,0.0,4.000000,0.981603,1
602,1061921690009,2,5c271cbd2d535234640e7e20,5ace203d9fe7b01d5806c08b,5c2995292d535234640e8681,1,75.350,Top10,0.496548,1.055505,0.250689,4.0,1.0,0.0,4.000000,0.981884,1


In [86]:
df[["Response","Predicted Responses"]]

,Response,Predicted Responses
0,2,1
1,1,1
2,1,1
3,2,0
4,1,0
...,...,...
599,1,1
600,1,1
601,1,1
602,1,1


In [16]:
#[j for j in range(25) if j%2==0]

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]

In [39]:
ability.columns[5].split('_')[0]

'1011211700004'

In [58]:
enrl = list(map(lambda sub: sub.split('_')[0], ability.columns[1:]))
cohort_en = list(map(lambda sub: sub.split('_')[1], ability.columns[1:]))
dct={'Enrolment No': enrl,'Cohort':cohort_en}
df_en_cohort = pd.DataFrame(dct)
df_en_cohort

,Enrolment No,Cohort
0,1011101700001,Top 1000
1,1011101700004,Top 500
2,1011211700001,Top 2000
3,1011211700003,Top 25000
4,1011211700004,Top 15000
...,...,...
7628,9065831590001,Top 20000
7629,9065831590002,Top 10000
7630,9065831590003,Top 10000
7631,9071101590001,Top 5000


['1011101700001',
 '1011101700004',
 '1011211700001',
 '1011211700003',
 '1011211700004',
 '1011221700003',
 '1011221700006',
 '1011291700002',
 '1011331700001',
 '1011401700001',
 '1011401700006',
 '1011401700011',
 '1011401700012',
 '1011421700004',
 '1011421700008',
 '1011421700014',
 '1011441700003',
 '1011451700001',
 '1011451700002',
 '1011451700004',
 '1011471700001',
 '1011471700002',
 '1011471700004',
 '1011471700006',
 '1011471700008',
 '1011471700010',
 '1011471700011',
 '1011471700019',
 '1011471700024',
 '1011471700026',
 '1011481700003',
 '1011481700006',
 '1011481700012',
 '1011481700013',
 '1011481700014',
 '1011521700003',
 '1011541690001',
 '1011541690008',
 '1011541690010',
 '1011541690011',
 '1011541690012',
 '1011541690015',
 '1011541700003',
 '1011541700004',
 '1011541700016',
 '1011641580001',
 '1011921700002',
 '1012101700002',
 '1012121700002',
 '1012201700002',
 '1012201700003',
 '1012511690002',
 '1012511690003',
 '1012511690005',
 '1012511700001',
 '10125117